**Input**:
    - pred_points: predicted list of 3d coordinates that are points along the neuron curve
    - ann_points: annotated list of 3d coordinates that are points along the neuron curve
        - of the form: (x, y, z, i) where i is the index of that point along the curve, with -1 indicating start of a curve
        
**Pseudocode**:
1. Iterate through the annotated neural tracts and calculate the lengths of each of the tracts.
2. Sum the all the lengths of the annotated neural tracts for that subvolume.
3. Repeat steps 1 and 2 for predicted neural tracts.
4. Take the difference between the sums of the lengths of the predicted and annotated neural tracts.
        

In [ ]:
get_difference(pred_points, ann_points):
    pred_total_dist = 0
    ann_total_dist = 0
    
    current_curve_distance = 0
    for i in xrange(1, len(ann_points)):
        if ann_points[i, 3] == -1:
            # if the next point is part of a different curve
            ann_total_dist += current_curve_distance
            current_curve_distance = 0
        current_curve_distance += euclidian_distance(ann_points[i-1, 0:4], ann_points[i, 0:4])
        
    current_curve_distance = 0
    for i in xrange(1, len(pred_points)):
        if pred_points[i, 3] == -1:
            # if the next point is part of a different curve
            pred_total_dist += current_curve_distance
            current_curve_distance = 0
        current_curve_distance += euclidian_distance(pred_points[i-1, 0:4], pred_points[i, 0:4])
    
    return (pred_total_dist - ann_total_dist)